In [1]:
import os
import requests
from typing import Dict, List, Optional
from pathlib import Path
from dotenv import load_dotenv
from tavily import TavilyClient
import json
import time

# Load environment variables
script_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
load_dotenv(script_dir / "SERPAPI_API_KEY.env")
load_dotenv(script_dir / "TAVILY_API_KEY.env")

SERPAPI_KEY = os.getenv("SERPAPI_API_KEY")
TAVILY_KEY = os.getenv("TAVILY_API_KEY")

# Validate API keys
if not SERPAPI_KEY:
    raise ValueError("⚠️ SERPAPI_API_KEY not found in environment file")
if not TAVILY_KEY:
    raise ValueError("⚠️ TAVILY_API_KEY not found in environment file")

print(f"SERPAPI_KEY loaded: {'Yes' if SERPAPI_KEY else 'No'}")
print(f"TAVILY_KEY loaded: {'Yes' if TAVILY_KEY else 'No'}")
print("="*60)

# Initialize Tavily client
tavily_client = TavilyClient(TAVILY_KEY)

def serpapi_search(query: str, num_results: int = 10, location: str = "United States") -> Dict:
    """
    Perform search using SerpAPI
    """
    url = "https://serpapi.com/search"
    params = {
        'api_key': SERPAPI_KEY,
        'engine': 'google',
        'q': query,
        'num': num_results,
        'location': location,
        'hl': 'en',
        'gl': 'us'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"SerpAPI Error: {e}")
        return {}

def tavily_search_simple(query: str, max_results: int = 10) -> Dict:
    """
    Perform simple search using Tavily
    """
    try:
        response = tavily_client.search(
            query=query,
            max_results=max_results
        )
        return response
    except Exception as e:
        print(f"Tavily Simple Search Error: {e}")
        return {}

def tavily_search_advanced(query: str, max_results: int = 10, 
                          include_domains: List[str] = None, 
                          exclude_domains: List[str] = None) -> Dict:
    """
    Perform advanced search using Tavily with filters
    """
    try:
        search_params = {
            'query': query,
            'search_depth': 'advanced',
            'max_results': max_results,
            'include_answer': True
        }
        
        if include_domains:
            search_params['include_domains'] = include_domains
        if exclude_domains:
            search_params['exclude_domains'] = exclude_domains
            
        response = tavily_client.search(**search_params)
        return response
    except Exception as e:
        print(f"Tavily Advanced Search Error: {e}")
        return {}

def format_serpapi_results(results: Dict) -> None:
    """
    Format and display SerpAPI results
    """
    print("🔍 SERPAPI RESULTS:")
    print("-" * 40)
    
    organic_results = results.get('organic_results', [])
    if not organic_results:
        print("No organic results found")
        return
    
    for i, result in enumerate(organic_results[:5], 1):  # Show top 5
        print(f"{i}. {result.get('title', 'No title')}")
        print(f"   URL: {result.get('link', 'No URL')}")
        print(f"   Snippet: {result.get('snippet', 'No snippet')}")
        print()

def format_tavily_results(results: Dict, search_type: str = "Simple") -> None:
    """
    Format and display Tavily results
    """
    print(f"🔍 TAVILY {search_type.upper()} RESULTS:")
    print("-" * 40)
    
    # Show generated answer if available
    if 'answer' in results:
        print("📝 Generated Answer:")
        print(results['answer'])
        print()
    
    search_results = results.get('results', [])
    if not search_results:
        print("No results found")
        return
    
    for i, result in enumerate(search_results[:5], 1):  # Show top 5
        print(f"{i}. {result.get('title', 'No title')}")
        print(f"   URL: {result.get('url', 'No URL')}")
        print(f"   Content: {result.get('content', 'No content')[:150]}...")
        print(f"   Score: {result.get('score', 'N/A')}")
        print()

def compare_searches(query: str):
    """
    Perform searches on both platforms and compare results
    """
    print(f"🔍 SEARCHING: '{query}'")
    print("="*60)
    
    # SerpAPI Search
    print("Searching with SerpAPI...")
    serpapi_results = serpapi_search(query, num_results=5)
    format_serpapi_results(serpapi_results)
    
    time.sleep(1)  # Small delay between API calls
    
    # Tavily Simple Search
    print("Searching with Tavily (Simple)...")
    tavily_simple_results = tavily_search_simple(query, max_results=5)
    format_tavily_results(tavily_simple_results, "Simple")
    
    time.sleep(1)  # Small delay between API calls
    
    # Tavily Advanced Search
    print("Searching with Tavily (Advanced)...")
    tavily_advanced_results = tavily_search_advanced(
        query, 
        max_results=5
        # Removed include_domains temporarily to test
    )
    format_tavily_results(tavily_advanced_results, "Advanced")
    
    print("="*60)

def main():
    """
    Main function to run search comparisons
    """
    # Example searches
    queries = [
        "Flights to Cancun Mexico from San Francisco from November 10 to November 20",
        "Recommendation for hotels and car rentals in Cancun",
        "Recommendations for points of interest in Cancun",
    ]
    
    for i, query in enumerate(queries, 1):
        print(f"\n{'='*20} SEARCH {i} {'='*20}")
        compare_searches(query)
        
        if i < len(queries):
            print("\n" + "🔄 Next search in 3 seconds...")
            time.sleep(3)

if __name__ == "__main__":
    main()

SERPAPI_KEY loaded: Yes
TAVILY_KEY loaded: Yes

==================== SEARCH 1 ====================
🔍 SEARCHING: 'Flights to Cancun Mexico from San Francisco from November 10 to November 20'
Searching with SerpAPI...
🔍 SERPAPI RESULTS:
----------------------------------------
1. Find Cheap Flights from San Francisco to Cancún (SFO
   URL: https://www.google.com/travel/flights/flights-from-san-francisco-to-cancun.html?gl=US&hl=en-US
   Snippet: Use Google Flights to find cheap flights from San Francisco to Cancún, starting at $410, and book tickets for your next trip.

2. Cheap Flights from San Francisco to Cancun (SFO-CUN)
   URL: https://www.expedia.com/lp/flights/sfo/cun/san-francisco-to-cancun
   Snippet: Cheap Flights from San Francisco (SFO) to Cancun (CUN) start at $184 for one-way and $366 for round trip. Earn your airline miles on top of our rewards!

3. Cheap Flights from San Francisco to Cancun
   URL: https://www.united.com/en-us/flights-from-san-francisco-to-cancun
   Snippe